In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'C:\\Users\\Param\\Python-Project\\CampusX\\GitHub\\end_to_end-ML_Project_with_MLFLOW'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/param-unik/end_to_end-ML_Project_with_MLFLOW.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="param-unik"
os.environ["MLFLOW_TRACKING_PASSWORD"]="3d3a1a07fbf32e2de561ef8dc2ca04e8fb341be4"

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path 
    model_path: Path
    all_params: dict 
    metric_file_name: Path 
    target_column: str 
    mlflow_uri: str 
    

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config_filepath = read_yaml(config_filepath)
        self.params_filepath = read_yaml(params_file_path)
        self.schema_filepath = read_yaml(schema_file_path)

        create_directories([self.config_filepath.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config_filepath.model_evaluation
        params = self.params_filepath.ElasticNet
        schema = self.schema_filepath.TARGET_COLUMNS 
        
        create_directories([config.root_dir])
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/param-unik/end_to_end-ML_Project_with_MLFLOW.mlflow"   
        )
        return model_evaluation_config
    

In [11]:
from mlProject.utils.common import save_json
import os
import pandas as pd 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib 

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, predicted):
        rmse = np.sqrt(mean_squared_error(actual, predicted))
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)
        
        return rmse, mae, r2 
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            
            prediction = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, prediction)
            
            scores = {'rmse': rmse, 'mae': mae, 'r2': r2}
            save_json(path=Path(self.config.metric_file_name), data = scores)
            
            
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name = "ElasticModel")
            else:
                mlflow.log_artifact(model, "model")

In [12]:
try:
    config_manager = ConfigurationManager()
    model_evaluation_config = config_manager.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e 
    

[2023-12-21 12:18:40,959: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-21 12:18:40,961: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-21 12:18:40,963: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-21 12:18:40,964: INFO: common: created directory at: artifacts]
[2023-12-21 12:18:40,965: INFO: common: created directory at: artifacts/model_evaluation]
[2023-12-21 12:18:41,499: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2023/12/21 12:18:45 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Param\AppData\Local\Temp\tmpq8n1gr0r\model\model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.3.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.
C:\Users\Param\Python-Project\CampusX\GitHub\end_to_end-ML_Project_with_MLFLOW\venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ElasticModel'.
2023/12/21 12:18:50 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticModel, version 1
Created version '1' of model 'ElasticModel'.
